In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
img_size = 150
labels = ['PNEUMONIA', 'NORMAL']
data_dir = '../../chest_xray/train'

# Function to load data
def load_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

train_data = load_data(data_dir)

# Separate features and labels
def separate_features_labels(data):
    features, labels = [], []
    for feature, label in data:
        features.append(feature)
        labels.append(label)
    return np.array(features), np.array(labels)

x_train, y_train = separate_features_labels(train_data)

# Normalize the data
x_train = x_train / 255.0
x_train = x_train.reshape(-1, img_size, img_size, 1)

# Data augmentation - flip
datagen = ImageDataGenerator(
    horizontal_flip=True
)

datagen.fit(x_train)

# Display some augmented images
def display_augmented_images(datagen, x_train, y_train):
    plt.figure(figsize=(10,10))
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=5):
        for i in range(0, 5):
            plt.subplot(330 + 1 + i)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(x_batch[i].reshape(img_size, img_size), cmap='gray')
        break
    plt.show()

display_augmented_images(datagen, x_train, y_train)

print("Flip augmentation complete.")
